# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [81]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [82]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [83]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [84]:
quantity = data.groupby(["CustomerID","ProductName"])["Quantity"].agg("sum")
quantity = pd.DataFrame(quantity)
quantity

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

In [85]:
import this

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [86]:
quant = pd.pivot_table(quantity, values = "Quantity", index = "ProductName", columns = ["CustomerID"], aggfunc= np.sum, fill_value = 0)
quant


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [87]:
from scipy.spatial.distance import pdist, squareform
distances = pdist(quant.T, "euclidean")
Q = squareform(distances)

distances = pd.DataFrame(1/(1+Q), index=quant.columns,columns=quant.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [88]:
similarities = distances[33].sort_values(ascending=False)[1:6]
similarities
listcust = list(similarities.index)
listcust

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [89]:
quantity

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

In [90]:
total = quantity.loc[listcust]
total

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [91]:
sumquantity = total.groupby(["ProductName"]).agg({"Quantity":"sum"}).sort_values("Quantity", ascending=False)
sumquantity

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [92]:
merged = quant.merge(sumquantity, left_on = "ProductName", right_on = "ProductName")
merged5= merged["Quantity"][merged[33]== 0].sort_values(ascending=False)
list(merged5[:5].index)


['Wine - Blue Nun Qualitatswein',
 'Butter - Unsalted',
 'Soup - Campbells Bean Medley',
 'Wine - Ej Gallo Sierra Valley',
 'Yoghurt Tubes']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [93]:
listcust = [i for i in data["CustomerID"]]
custonce = (set(listcust))


In [97]:
def custompref(custonce):
    diction = {}
    for i in custonce:
        similarities = distances[i].sort_values(ascending=False)[1:6]
        listcust = list(similarities.index)
        total = quantity.loc[listcust]
        sumquantity = total.groupby(["ProductName"]).agg({"Quantity":"sum"}).sort_values("Quantity", ascending=False)
        merged = quant.merge(sumquantity, left_on = "ProductName", right_on = "ProductName")
        merged5= merged["Quantity"][merged[i]== 0].sort_values(ascending=False)
        diction[i]=list(merged5[:5].index)
    return diction
#custompref(custonce)


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [98]:
pref = custompref(custonce)
preferences = pd.DataFrame.from_dict(pref)
preferences.head()

,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Cocoa Butter,"Shrimp - Baby, Warm Water",Cup - Translucent 7 Oz Clear,"Pasta - Penne, Rigate, Dry",Soupfoamcont12oz 112con,Bacardi Breezer - Tropical,Spinach - Baby,Blueberries,Guinea Fowl,Cookies - Assorted,...,"Pepper - Paprika, Hungarian",Ice Cream Bar - Oreo Cone,Sea Bass - Whole,Cod - Black Whole Fillet,Olive - Spread Tapenade,Cheese - Victor Et Berthold,Grouper - Fresh,"Pasta - Detalini, White, Fresh",Cake - Mini Cheesecake,Sauce - Rosee
1,Veal - Slab Bacon,Towels - Paper / Kraft,"Veal - Brisket, Provimi,bnls",Cookies - Assorted,"Lemonade - Natural, 591 Ml",Cheese - Mozzarella,Vinegar - Tarragon,Turkey - Oven Roast Breast,Ecolab - Lime - A - Way 4/4 L,Wine - Redchard Merritt,...,V8 - Berry Blend,Vol Au Vents,Spinach - Baby,Thermometer Digital,Pants Custom Dry Clean,Tilapia - Fillets,Sherry - Dry,Soup - Campbells Tomato Ravioli,Ecolab - Solid Fusion,Scallop - St. Jaques
2,Bandage - Fexible 1x3,Knife Plastic - White,Snapple Lemon Tea,Cheese - Parmesan Grated,Soup Knorr Chili With Beans,Banana - Leaves,Pork - Hock And Feet Attached,Mussels - Cultivated,Bread Crumbs - Panko,Juice - Orange,...,Soupcontfoam16oz 116con,Soupcontfoam16oz 116con,Ice Cream Bar - Oreo Cone,"Pork - Back, Short Cut, Boneless",Sword Pick Asst,Cake - Cake Sheet Macaroon,"Chestnuts - Whole,canned","Salsify, Organic",Wine - Redchard Merritt,"Chestnuts - Whole,canned"
3,Ezy Change Mophandle,Nantucket - Pomegranate Pear,Bread - Calabrese Baguette,Puree - Mocha,Sausage - Breakfast,Table Cloth - 53x69 Colour,Bread Foccacia Whole,Vaccum Bag 10x13,Bread - English Muffin,Juice - V8 Splash,...,Brandy - Bar,Ketchup - Tomato,Beets - Mini Golden,Sponge Cake Mix - Chocolate,Arizona - Green Tea,Macaroons - Two Bite Choc,Napkin White - Starched,Wanton Wrap,Mussels - Frozen,"Bread - Roll, Soft White Round"
4,Liners - Baking Cups,Bagel - Plain,Spinach - Baby,Rice - Jasmine Sented,Sprouts - Baby Pea Tendrils,Chocolate - Dark,Thermometer Digital,Bread - Calabrese Baguette,Sunflower Seed Raw,Table Cloth 54x72 White,...,Vinegar - Tarragon,Cheese - Taleggio D.o.p.,Anchovy Paste - 56 G Tube,Beans - Wax,Pail For Lid 1537,Wine - Ruffino Chianti,Yoghurt Tubes,"Wine - White, Colubia Cresh",Thermometer Digital,Chicken - Wieners


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [99]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import pearsonr
C2 = np.corrcoef(quant.values.T)
correldistance = pd.DataFrame(C2)
correldistance


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,-0.025211,0.112057,0.040600,0.027421,0.028458,-0.051663,-0.008305,-0.003146,0.009948,...,-0.026814,-0.064510,-0.026400,-0.083910,-0.000520,-0.084359,-0.027589,-0.055680,-0.035068,-0.032223
1,-0.025211,1.000000,-0.001067,-0.005715,-0.073948,-0.075028,0.073735,-0.080738,-0.003482,0.021601,...,0.018700,-0.015439,0.069067,0.035186,-0.030125,-0.063924,0.129062,0.001010,-0.064954,-0.046097
2,0.112057,-0.001067,1.000000,-0.009730,0.052856,-0.039546,0.012890,-0.079281,0.008078,-0.039807,...,0.034602,-0.037471,-0.001002,-0.003357,0.068539,0.096246,0.046285,0.013902,0.061024,-0.037179
3,0.040600,-0.005715,-0.009730,1.000000,-0.040222,-0.041743,0.007702,0.030732,-0.071740,-0.056114,...,-0.003586,-0.054821,-0.005365,0.008873,-0.023312,-0.015781,-0.065232,0.008777,0.025281,0.002520
4,0.027421,-0.073948,0.052856,-0.040222,1.000000,-0.009738,0.020238,0.048666,-0.024509,0.030633,...,-0.048723,0.111432,-0.032385,-0.019126,0.018828,0.000255,-0.077885,0.009625,0.023704,0.058310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.084359,-0.063924,0.096246,-0.015781,0.000255,0.031355,0.018780,-0.002805,0.001072,-0.029966,...,-0.021932,0.018594,-0.050846,0.039427,0.129001,1.000000,0.097349,0.035065,-0.032316,-0.070734
996,-0.027589,0.129062,0.046285,-0.065232,-0.077885,-0.065183,-0.012509,-0.039542,-0.018858,0.023863,...,-0.043515,-0.001031,-0.027032,-0.063468,0.069590,0.097349,1.000000,0.045758,0.004244,-0.004420
997,-0.055680,0.001010,0.013902,0.008777,0.009625,-0.003224,0.000960,0.008435,0.040495,0.008652,...,0.020094,0.086952,0.002208,-0.044248,0.025048,0.035065,0.045758,1.000000,0.017318,-0.017836
998,-0.035068,-0.064954,0.061024,0.025281,0.023704,-0.042204,-0.049346,0.043575,0.000377,-0.083135,...,-0.067277,-0.125484,-0.009437,-0.020435,-0.040386,-0.032316,0.004244,0.017318,1.000000,0.023378


In [100]:
listcust = [i for i in data["CustomerID"]]
custonce2 = (set(listcust))
def custompref(custonce2):
    diction = {}
    for i in custonce:
        similarities = correldistance[i].sort_values(ascending=False)[1:6]
        listcust = list(similarities.index)
        total = quantity.loc[listcust]
        sumquantity = total.groupby(["ProductName"]).agg({"Quantity":"sum"}).sort_values("Quantity", ascending=False)
        merged = quant.merge(sumquantity, left_on = "ProductName", right_on = "ProductName")
        merged5= merged["Quantity"][merged[i]== 0].sort_values(ascending=False)
        diction[i]=list(merged5[:5].index)
    return diction

In [101]:
custompref(custonce2)

KeyError: 83973

In [102]:
from sklearn.metrics.pairwise import cosine_similarity as distance
similarity_matrix = pdist(quant.T, "cosine")
distancecos= pd.DataFrame(similarity_matrix)
distancecos

,0
0,0.886781
1,0.774648
2,0.839772
3,0.843681
4,0.848243
...,...
499495,0.844531
499496,0.863140
499497,0.840744
499498,0.881655


In [103]:
listcust = [i for i in data["CustomerID"]]
custonce3 = (set(listcust))
def custompref(custonce3):
    diction = {}
    for i in custonce:
        similarities = distancecos[i].sort_values(ascending=False)[1:6]
        listcust = list(similarities.index)
        total = quantity.loc[listcust]
        sumquantity = total.groupby(["ProductName"]).agg({"Quantity":"sum"}).sort_values("Quantity", ascending=False)
        merged = quant.merge(sumquantity, left_on = "ProductName", right_on = "ProductName")
        merged5= merged["Quantity"][merged[i]== 0].sort_values(ascending=False)
        diction[i]=list(merged5[:5].index)
    return diction

In [79]:
custompref(custonce3)

KeyError: 83973